In [1]:
#################### Feature-Extraktion Code ###################

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
import statistics 
import neurokit2 as nk #peak detect from ecg and ppg usw

record =[]
resultsPTT = []
resultsPTTs = []
resultsPTTv =[]
HR =[]
BP_sys =[]
BP_dia = []
PTTstd = []
PTTsstd = []
PTTvstd = []

subject_info = pd.read_csv("PhysioNet_PTT_Dataset/subjects_info.csv")
personal_data = pd.DataFrame([])

time_span = 5000 #10 seconds anlayse

#schlechte Daten werden herausgenommen
notgoodWalkStart = [2,7,8,10,11,12,13,14,19,20]
notgoodWalkEnd = [1,2,4,6,7,8,9,10,11,12,13,14,15,16,19,20]
notgoodRunStart = [1,2,6,8,9,11,12,19]   #12 the valley is not good 
notgoodRunEnd = [2,8,9,11,12,17,19,20]  #17 is actually okay but needs a additional filter 


for j in range(3):
    for k in range(2):   # first goes for start 1-22 and then end 1-22 
     #Iterate from 1 to 22
        for i in range(1, 23):
            

            if j == 0:
                action = "sit"
            elif j ==1:
                action = "walk"
            else:
                action = "run"

            while action == "walk" and k==0 and i in notgoodWalkStart:
                i += 1
            while action == "walk" and k==1 and i in notgoodWalkEnd:
                i += 1
            while action == "run" and k==0 and i in notgoodRunStart:
                i += 1
            while action == "run" and k==1 and i in notgoodRunEnd:
                i += 1
                
            df = pd.read_csv("PhysioNet_PTT_Dataset/s" + str(i) + "_" + action +".csv")    

            if k ==0:
                pos = "start"
                df_neu = df[0:time_span]
            else:
                pos = "end"
                df_neu = df.tail(time_span)

            uno = df_neu['pleth_1']
            dos = df_neu['pleth_2']
            tres= df_neu['pleth_3']

            quatro = df_neu['pleth_4']
            cinco = df_neu['pleth_5']
            seis = df_neu['pleth_6']
            # use the zip() function to combine the arrays
            combinedDist= zip(uno, dos ,tres)
            combinedProx = zip(quatro, cinco, seis)

            # use a list comprehension to calculate the median of each triplet of elements
            meanDist = [ (x+y+z)/3 for x, y, z in combinedDist ]

            # Finde Peak / Maximum of PPG 
            signals, info = nk.ppg_process(dos, sampling_rate=500)
            PeaksPPG = info['PPG_Peaks']

            # Find steepest Ascent 
            steepestAscent= -np.diff(signals['PPG_Clean'])   #derivation of the signal
            siganlsStepAsc, infoS = nk.ecg_process(steepestAscent, sampling_rate=500)  # finding the peaks of the derivation therefore the steepest ascent
            SteepAsc = infoS['ECG_R_Peaks']

            # Find valley / Minimum of PPG
            mirrored_PPG_raw = np.array(dos)
            mirrored_PPG_raw = (1/mirrored_PPG_raw)*max(mirrored_PPG_raw)# mirroring the signal an normalizing to 1)
            signalsValleys, inf = nk.ppg_process(mirrored_PPG_raw, sampling_rate=500)
            ValleyPPG = inf['PPG_Peaks']

            SignalPeaksECG, ECGInfo = nk.ecg_process(df_neu["ecg"],sampling_rate=500)
            #PeaksECG = ECGInfo["ECG_R_Peaks"]
            p = df_neu['peaks']
            PeaksECG = p[p==1]
            PeaksECG = np.array(PeaksECG.index)

            if PeaksECG[1]> 100000:    #if end of data set is observed then we need to zero the data to calculate
                PeaksECG = PeaksECG-(len(df)-time_span)
        
            #Plot that shit
            print("Person: ",i)
            print("Action:",action)
            if k == 0:
                print("Start")
            else: 
                print("End")

            #changing dataframe so that peaks are on the ppg signal as a cross 
            pPPG = signals['PPG_Peaks']
            pSAc = siganlsStepAsc['ECG_R_Peaks']
            pV   = signalsValleys['PPG_Peaks']

            pPPG.loc[signals['PPG_Peaks'] == 1] = signals['PPG_Clean']
            pSAc.loc[siganlsStepAsc['ECG_R_Peaks'] == 1] = signals['PPG_Clean']
            pV.loc[signalsValleys['PPG_Peaks'] == 1] = signals['PPG_Clean']
            
            pPPG = pPPG.loc[pPPG != 0].to_frame()
            pSAc= pSAc.loc[pSAc != 0].to_frame()
            pV= pV.loc[pV != 0].to_frame()

            #plot that shit
            plt.plot(signals['PPG_Clean'])
            plt.plot(np.arange(0, time_span),df_neu['peaks']*500, label="ECG Peaks")     #SignalPeaksECG['ECG_R_Peaks']
            plt.scatter(pPPG.index,pPPG,label="PPG Peaks",marker = "x")
            plt.scatter(pSAc.index, pSAc,label="PPG Steepest Ascent",marker = "x")
            plt.scatter(pV.index, pV,label="PPG: Valleys",marker = "x")
            plt.legend()
            plt.show()

            
            print("PeaksECG:", PeaksECG)
            print("PeaksPPG:", PeaksPPG)
            print("SteepAsc:", SteepAsc)
            print("ValleyPPG:", ValleyPPG)


            #a little bit of filtering ect to detect only the peaks which are relevant
            if PeaksECG[0] > PeaksPPG[0]:
                PeaksPPG = np.delete(PeaksPPG,[0])

            elif  PeaksPPG[0] - PeaksECG[0] >260:
                PeaksECG = np.delete(PeaksECG,[0])

            if PeaksPPG[0] > SteepAsc[0]:
                SteepAsc = np.delete(SteepAsc,[0])
            
            if SteepAsc[0] > ValleyPPG[0]:
                ValleyPPG = np.delete(ValleyPPG,[0])

            print("PeaksECG Korr.:", PeaksECG)  
            print("PeaksPPG Korr.:", PeaksPPG)
            print("SteepASc Korr.:", SteepAsc)
            print("ValleyPPG Korr.:", ValleyPPG)
            
            PTT = np.array([x - y for x, y in zip(PeaksPPG, PeaksECG)])   #PTT =  PeaksPPG - PeaksECG
            print("PTT:", PTT)
        #delete the outliers of PTT
            mean = np.mean(PTT)
            standard_deviation = np.std(PTT)
            distance_from_mean = abs(PTT - mean)
            max_deviations = 2
            not_outlier = distance_from_mean < max_deviations * standard_deviation
            PTT_no_outliers = PTT[not_outlier]
            #print("PTT_no_outliers: ", PTT_no_outliers)

            PTTs = np.array([x - y for x, y in zip(SteepAsc, PeaksECG)])
            print("PTTs:", PTTs)
        #delete the outliers of PTTs
            mean = np.mean(PTTs)
            standard_deviation = np.std(PTTs)
            distance_from_mean = abs(PTTs - mean)
            max_deviations = 2
            not_outlier = distance_from_mean < max_deviations * standard_deviation
            PTTs_no_outliers = PTTs[not_outlier]
            #print("PTTs_no_outliers: ", PTTs_no_outliers)

            PTTv =  np.array([x - y for x, y in zip(ValleyPPG, PeaksECG)])
            print("PTTv:", PTTv)
        #delete the outliers of PTTv
            mean = np.mean(PTTv)
            standard_deviation = np.std(PTTv)
            distance_from_mean = abs(PTTv - mean)
            max_deviations = 2
            not_outlier = distance_from_mean < max_deviations * standard_deviation
            PTTv_no_outliers = PTTv[not_outlier]
            #print("PTTv_no_outliers: ",PTTv_no_outliers)



            print("Mean PTT (-outliers):",statistics.mean(PTT_no_outliers))
            print("Mean PTTs (-outliers):",statistics.mean(PTTs_no_outliers))
            print("Mean PTTv (-outliers):",statistics.mean(PTTv_no_outliers))
            print("Heart Rate HR:", statistics.mean(SignalPeaksECG["ECG_Rate"]))


        #append peak-positions to list
            resultsPTT = np.append(resultsPTT, statistics.mean(PTT_no_outliers))
            PTTstd = np.append(PTTstd, np.std(PTT_no_outliers))
            resultsPTTs = np.append(resultsPTTs, statistics.mean(PTTs_no_outliers))
            PTTsstd = np.append(PTTsstd, np.std(PTTs_no_outliers))
            resultsPTTv = np.append(resultsPTTv, statistics.mean(PTTv_no_outliers))
            PTTvstd = np.append(PTTvstd, np.std(PTTv_no_outliers))
            HR = np.append(HR, statistics.mean(SignalPeaksECG["ECG_Rate"]))
            record = np.append(record, "s" + str(i) + "_" + action + "_" + pos)


            subject_row = subject_info[subject_info['record'] == "s"+ str(i)+ "_" + action]    #takes the current row of the for loop

            if pos == "start":
                BP_sys = np.append(BP_sys, subject_row["bp_sys_start"])
                BP_dia = np.append(BP_dia, subject_row["bp_dia_start"])

            elif pos =="end":
                BP_sys = np.append(BP_sys, subject_row["bp_sys_end"])
                BP_dia = np.append(BP_dia, subject_row["bp_dia_end"])


            personal_data = personal_data.append(subject_row, ignore_index=True)  # adds the row to the 

ResultsFrame = pd.DataFrame({"record": record, 'PTTp': resultsPTT, "PTTs":resultsPTTs, "PTTv":resultsPTTv, "HR":HR, "BP_sys":BP_sys, "BP_dia":BP_dia, "PTTp std": PTTstd,"PTTs std": PTTsstd,"PTTv std": PTTvstd,})
# Add two columns from df1 (A and B) to df2
ResultsFrame = ResultsFrame.join(personal_data[['activity', 'gender','height', 'weight','age']])
ResultsFrame = ResultsFrame.drop_duplicates(subset=["record", "PTTp","HR", "BP_dia"])  # deletes doublicates

ResultsFrame = ResultsFrame[ResultsFrame['record'] != 's11_sit_start']  # 11 ist no good 
ResultsFrame = ResultsFrame[ResultsFrame['record'] != 's11_sit_end']

# One-hot encode the gender variable
ResultsFrame = pd.get_dummies(ResultsFrame, columns=['gender', 'activity'])

ResultsFrame.to_csv('Results1.txt', sep='\t', index=False)



ModuleNotFoundError: No module named 'neurokit2'